**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /data/reddylab/Kuei
WORK DIRECTORY:     /data/reddylab/Kuei/out
CODE DIRECTORY:     /data/reddylab/Kuei/code
SING DIRECTORY:     /data/reddylab/Kuei/singularity
PATH OF SOURCE:     /data/reddylab/Kuei/source
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc



## Intersect with peaks

In [6]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/intersect_peaks_rolling_correlation.%a.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.peaks.MYC.FADS.GATA1.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/results/comparison/comparison_local/cor_size_200bp_step_50bp/summary
FN_ANN=cor.pLog2FC.combine.bed
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_ANN}
FN_OUT=cor.pLog2FC.combine.windows.bed.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
cat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_BED} \
    -b ${FP_ANN} \
    -wo \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29434171


In [7]:
cat ${FD_LOG}/intersect_peaks_rolling_correlation.0.txt

Hostname:           x1-02-1.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         12-20-22+11:55:54

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.peaks.MYC.FADS.GATA1.bed

show first few lines of input
chr8	126778902	126779728	chr8_126778902_126779728	MYC
chr8	126782925	126783318	chr8_126782925_126783318	MYC
chr8	126804776	126805825	chr8_126804776_126805825	MYC
chr8	126817018	126817490	chr8_126817018_126817490	MYC
chr8	126820465	126821393	chr8_126820465_126821393	MYC

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/comparison/comparison_local/cor_size_200bp_step_50bp/summary/cor.pLog2FC.combine.bed

show first few lines of input
chr8	126735901	126736101	126736001	0.296774023401668	-0.926031965171654	-0.268755028923739
chr8	126735951	126736151	126736051	-0.718753798422289	-0.927141285349486	0.775338650893125
chr8	126736001	126736201	126736101	-0.743443972036306	-0.869901958520832	0.8